### Author: Kyle Tranfaglia ###
### Course: DSCI 470 ###
### Assignment: The Project ###
### Last Updated: 10/23/24 ###

## Background: ##

## Questions: ## 
### How do opening choices and event types differ across Elo ratings, and are certain openings more likely in specific events or Elo ranges? ###
### How do rating differences and player Elo impact game outcomes and the predictability of the game results? ### 
### Which game features (event type, opening code, result) have the most predictive power for Elo, and how can these insights inform match-making criteria? ###

## Prepare for Analysis: Import Libraries and Read in Data

In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read in data set and strip beginning spaces from the 'Event' column
df_chess = pd.read_csv("Data/Big_chess_data.csv", nrows=1000000, converters= {'Event': lambda x: x.strip() if isinstance(x, str) else x})


## Utility Functions ##

In [51]:
# Set up the plotting style
sns.set_theme(style='whitegrid')


# Function to plot bar charts
def plot_frequencies(counts, title, xlabel, ylabel):
    # Handle frequencies with too many categories to appropriately disaplay (n > 50)
    top_n = 50
    counts = counts.nlargest(top_n)

    plt.figure(figsize=(12, 8))
    sns.barplot(x=counts.index, y=counts.values, hue=counts.index, legend=False, palette='viridis')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.yscale('log')
    plt.xticks(rotation=90)  # Rotate x labels for better readability
    plt.show()


# Extract the first word of a column entry
def extract_first_word(entry):
    return entry.split()[0]


# Extract the first two words of a column entry
def extract_first_two_words(entry):
    words = entry.replace(":", "").split()  # Remove colons and split the string
    return ' '.join(words[:2]) if len(words) > 1 else words[0]


# Assign a player strength classification to an Elo
def classify_elo(elo):
    if elo <= 1200:
        return 'Beginner'
    elif elo <= 1800:
        return 'Intermediate'
    elif elo <= 2000:
        return 'Advanced'
    elif elo <= 2200:
        return 'Expert'
    else:
        return 'Master'

## Explore the Data ##

In [35]:
# Display some data, statistics, and summary info
print(df_chess.describe())
print("\n", df_chess.head())

# Create a new column with generalized openings (group all opening variations)
df_chess['generalized_Opening'] = df_chess['Opening'].apply(extract_first_two_words)

# Get and display data fequencies for some features
opening_counts = df_chess['generalized_Opening'].value_counts().sort_index()
print("\nFrequency of", opening_counts)
plot_frequencies(opening_counts, 'Frequencies of Openings', 'Opening', 'Count')

ECO_counts = df_chess['ECO'].value_counts().sort_index()
print("\nFrequency of", ECO_counts)
plot_frequencies(ECO_counts, 'Frequencies of ECO', 'ECO', 'Count')

time_control_counts = df_chess['TimeControl'].value_counts().sort_index()
print("\nFrequency of", time_control_counts)
plot_frequencies(time_control_counts, 'Frequencies of Time Control', 'Time Control', 'Count')

event_counts = df_chess['Event'].value_counts().sort_index()
print("\nFrequency of", event_counts)
plot_frequencies(event_counts, 'Frequencies of Events', 'Event', 'Count')

termination_counts = df_chess['Termination'].value_counts().sort_index()
print("\nFrequency of", termination_counts)
plot_frequencies(termination_counts, 'Frequencies of Terminations', 'Termination', 'Count')


df_chess[['BlackElo', 'WhiteElo']].plot(kind='hist', bins=50, alpha=0.5, title='Elo Distribution', figsize=(10, 8))

# Create the boxplot with subplots for White and Black Elo ratings
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Boxplot for WhiteElo
df_chess.boxplot(column='WhiteElo', by='Event', ax=axes[0])
axes[0].set_title('White Elo Distribution by Event')
axes[0].set_xlabel('Event')
axes[0].set_ylabel('White Elo')

# Boxplot for BlackElo
df_chess.boxplot(column='BlackElo', by='Event', ax=axes[1])
axes[1].set_title('Black Elo Distribution by Event')
axes[1].set_xlabel('Event')
axes[1].set_ylabel('Black Elo')

# Rotate x-axis labels for both subplots
for ax in axes:
    plt.sca(ax)
    plt.xticks(rotation=45, ha='right')

plt.suptitle('')  # Remove the automatic generated title
plt.show()  # Show the plot

                   Event   generalized_Opening  Frequency Percentage
0                  Blitz      Sicilian Defense      49206     13.13%
1                  Blitz        French Defense      27997      7.47%
2                  Blitz          Queen's Pawn      25640      6.84%
3                  Blitz        Queen's Gambit      17548      4.68%
4                  Blitz  Scandinavian Defense      17426      4.65%
5                  Blitz           King's Pawn      13402      3.58%
6                  Blitz     Caro-Kann Defense      12771      3.41%
7       Blitz tournament      Sicilian Defense      11808     15.58%
8       Blitz tournament        French Defense       5387      7.11%
9       Blitz tournament          Queen's Pawn       4988      6.58%
10      Blitz tournament  Scandinavian Defense       3640      4.80%
11      Blitz tournament     Caro-Kann Defense       3308      4.36%
12      Blitz tournament        Queen's Gambit       3264      4.31%
13      Blitz tournament       Eng

## Explore the Question: How do opening choices and event types differ across Elo ratings, and are certain openings more likely in specific events or Elo ranges? ##

In [55]:
'''Add four new columns to the dataset for improved analysis and efficency'''
# Create a new column for the Elo group of each player
df_chess['WhiteEloGroup'] = df_chess['WhiteElo'].apply(classify_elo)
df_chess['BlackEloGroup'] = df_chess['BlackElo'].apply(classify_elo)

# Calculate average Elo and Elo group based on the average Elo for each game and add to the DataFrame
df_chess['AvgElo'] = df_chess[['WhiteElo', 'BlackElo']].mean(axis=1)
df_chess['EloGroup'] = df_chess['AvgElo'].apply(classify_elo)

'''Calculate and display the average elo for black, white, and combined with a classification for the elo for each generalized opening'''
# Get the average elo of players that use each opening by grouping 'generalized_Opening' and 'WhiteElo' and 'BlackElo'
avg_opening_elo = df_chess.groupby('generalized_Opening')[['WhiteElo', 'BlackElo']].mean()

# Create a new column for combined average Elo & categorize each opening based on its average Elo
avg_opening_elo['AvgElo'] = avg_opening_elo.mean(axis=1)
avg_opening_elo['EloGroup'] = avg_opening_elo['AvgElo'].apply(classify_elo)
print(avg_opening_elo)

# Group the openings by the average Elo Group
openings_by_mean_elo_group = avg_opening_elo.groupby('EloGroup')

# Iterate through the groups and display the openings in each Elo Group
for group, openings in openings_by_mean_elo_group:
    print(f"\nElo Group: {group}")
    print(openings[['AvgElo']].sort_values(by='AvgElo', ascending=False))

'''Use the elo group frequencies for each opening and the total number of players in each elo group to normalize the mode of each 
opening within an elo group and display the most frequent elo group for each generalized opening'''

# Combine White and Black Elo groups into a single DataFrame for counting
elo_groups_combined = pd.concat([
    df_chess[['generalized_Opening', 'WhiteEloGroup']].rename(columns={'WhiteEloGroup': 'EloGroup'}),
    df_chess[['generalized_Opening', 'BlackEloGroup']].rename(columns={'BlackEloGroup': 'EloGroup'})
])

# Count the frequency of each Elo group for each opening & count the total number of players in each Elo group
elo_group_counts = elo_groups_combined.groupby(['generalized_Opening', 'EloGroup']).size().reset_index(name='Count')
total_players_by_elo_group = elo_group_counts.groupby('EloGroup')['Count'].sum()

# Normalize the frequency of openings within each Elo group by dividing by the total number of players in that group
# This is significant as the elo groups are inbalanced which would lead to bias
elo_group_counts['NormalizedCount'] = elo_group_counts.apply(
    lambda row: row['Count'] / total_players_by_elo_group[row['EloGroup']], axis=1
)

# For each opening, find the Elo group with the highest frequency
most_frequent_elo_group = (
    elo_group_counts.groupby('generalized_Opening', group_keys=False)
    .apply(lambda x: x.loc[x['NormalizedCount'].idxmax()])  # Get the row with the highest count for each opening
    .reset_index(drop=True)
)
print(most_frequent_elo_group)

# Group the openings by the most frequent Elo Group (from the normalized classification)
openings_by_mode_elo_group = most_frequent_elo_group.groupby('EloGroup')

# Iterate through the groups and display the openings in each Elo Group
for group, openings in openings_by_mode_elo_group:
    print(f"\nElo Group: {group}")
    print(openings[['generalized_Opening', 'NormalizedCount']].sort_values(by='NormalizedCount', ascending=False))

'''Calculate and display the top 5 most frequent openings played for each event with a percentage of the totol games that the opening was played'''
# Create a series by grouping 'Event' and frequencies of each 'generalized_Opening' within each event, then get the top 5 most frequent openings
top_openings_by_event = (
    df_chess.groupby('Event')['generalized_Opening']
    .value_counts()
    .groupby(level=0)
    .nlargest(5)
    .reset_index(level=0, drop=True)
)

top_openings_by_event = top_openings_by_event.reset_index(name='Frequency')  # Convert the Series to a DataFrame 
total_games_by_event = df_chess['Event'].value_counts()  # Get the total amount of games played for each event 

# Calculate the percentage of the total games that an opening was played for each event, and create a formatted column 
top_openings_by_event['Percentage'] = top_openings_by_event.apply(
    lambda row: f"{(row['Frequency'] / total_games_by_event[row['Event']]) * 100:.2f}%", axis=1
)
print(top_openings_by_event)

'''Calculate and display the average black, white, and combined elo for every event type'''
# Calculate the average Elo for each event type and create an additional column to represent the combined average Elo and an elo group
avg_elo_by_event = df_chess.groupby('Event')[['WhiteElo', 'BlackElo']].mean()
avg_elo_by_event['AvgElo'] = avg_elo_by_event.mean(axis=1)
avg_elo_by_event = avg_elo_by_event.rename(columns={
    'WhiteElo': 'Avg White Elo',
    'BlackElo': 'Avg Black Elo',
    'AvgElo': 'Avg Combined Elo'
})
avg_elo_by_event['Elo Classification'] = avg_elo_by_event['Avg Combined Elo'].apply(classify_elo)
print(avg_elo_by_event)

                              WhiteElo     BlackElo       AvgElo      EloGroup
generalized_Opening                                                           
Alekhine Defense           1780.893042  1798.162747  1789.527895  Intermediate
Amar Opening               1661.087156  1756.096330  1708.591743  Intermediate
Amazon Attack              1536.135922  1578.422330  1557.279126  Intermediate
Amsterdam Attack           1666.500000  1758.500000  1712.500000  Intermediate
Anderssen Opening          1655.411572  1655.634279  1655.522926  Intermediate
...                                ...          ...          ...           ...
Ware Opening               1588.562249  1671.676037  1630.119143  Intermediate
Yusupov-Rubinstein System  1811.565891  1832.612403  1822.089147      Advanced
Zaire Defense              2057.000000  1889.000000  1973.000000      Advanced
Zukertort Defense          1575.625000  1477.750000  1526.687500  Intermediate
Zukertort Opening          1808.161709  1790.342040 

/tmp/ipykernel_32237/744159578.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.loc[x['NormalizedCount'].idxmax()])  # Get the row with the highest count for each opening


           generalized_Opening      EloGroup  Count  NormalizedCount
0             Alekhine Defense        Master   1538         0.017521
1                 Amar Opening      Beginner     37         0.000987
2                Amazon Attack      Beginner     71         0.001894
3             Amsterdam Attack  Intermediate      3         0.000003
4            Anderssen Opening      Beginner     57         0.001520
..                         ...           ...    ...              ...
138               Ware Opening      Beginner     98         0.002614
139  Yusupov-Rubinstein System        Master    120         0.001367
140              Zaire Defense        Expert      1         0.000004
141          Zukertort Defense  Intermediate     15         0.000013
142          Zukertort Opening        Master   3579         0.040772

[143 rows x 4 columns]

Elo Group: Advanced
    generalized_Opening  NormalizedCount
100      Queen's Gambit         0.043776
55      Horwitz Defense         0.016388
89  